# Tâche 1 — Reconnaissance faciale robuste (Notebook unique)

Pipeline : lecture → détection/alignement → embeddings (modèle pré-entraîné) → **matching 1-à-1** → **export CSV**.


## 1) Installation (si nécessaire)
Décommente la cellule ci-dessous si les dépendances ne sont pas installées.


In [ ]:
%pip install --upgrade pip
%pip install insightface opencv-python numpy scipy scikit-learn networkx tqdm matplotlib pyyaml

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# pip install matplotlib
# 

  Using cached insightface-0.7.3-cp311-cp311-win_amd64.whl
  Using cached onnx-1.19.0-cp311-cp311-win_amd64.whl.metadata (7.2 kB)
  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached onnx-1.19.0-cp311-cp311-win_amd64.whl (16.5 MB)

   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 206] Nom de fichier ou extension trop long: 'C:\\Users\\kokou\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\onnx\\backend\\test\\data\\node\\test_attention_3d_with_past_and_present_qk_matmul_softcap_expanded\\test_data_set_0'



## 2) Imports & configuration
Charge le YAML de config, prépare les dossiers, fixe la seed.

In [1]:
import os, glob, csv, json
from pathlib import Path
import numpy as np
import cv2
import yaml
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, roc_curve
from scipy.optimize import linear_sum_assignment

# Chargement de la config
CONFIG_PATH = Path("configs/config.yaml")
assert CONFIG_PATH.exists(), "Fichier configs/config.yaml introuvable."
config = yaml.safe_load(open(CONFIG_PATH, "r", encoding="utf-8"))

DATA_DIR   = Path(config["paths"]["data_dir"])
TRAIN_DIR  = Path(config["paths"]["train_dir"])
TEST_DIR   = Path(config["paths"]["test_dir"])
OUT_DIR    = Path(config["paths"]["outputs_dir"])
FIG_DIR    = Path(config["paths"]["figures_dir"])

MODEL_NAME = config["model"]["insightface_name"]
CTX_ID     = int(config["model"]["ctx_id"])
DET_SIZE   = tuple(config["model"]["det_size"])

DEV_RATIO  = float(config["dev_eval"]["dev_ratio"])
SEED       = int(config["dev_eval"]["random_seed"])

EXPORT_NAME = config["matching"]["export_filename"]
SAVE_FIG    = bool(config["logging"]["save_figures"])

np.random.seed(SEED)
OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print("Config:", json.dumps(config, indent=2))


Config: {
  "paths": {
    "data_dir": "data",
    "train_dir": "data/train",
    "test_dir": "data/test",
    "outputs_dir": "outputs",
    "figures_dir": "outputs/figures"
  },
  "model": {
    "insightface_name": "buffalo_l",
    "ctx_id": 0,
    "det_size": [
      640,
      640
    ]
  },
  "dev_eval": {
    "dev_ratio": 0.2,
    "random_seed": 42
  },
  "matching": {
    "solver": "hungarian",
    "block_size": 0,
    "export_filename": "tache1_submission.csv"
  },
  "logging": {
    "verbose": true,
    "save_figures": true
  }
}


## 3) Détecteur + extracteur d'embeddings (InsightFace)
Utilise un backbone ArcFace pré-entraîné (`buffalo_l`). Retourne des embeddings **L2-normalisés** (512D).

In [ ]:
pip install numpy scipy scikit-learn matplotlib tqdm pyyaml
pip install opencv-python-headless
pip install "insightface>=0.7.3" onnxruntime-gpu

SyntaxError: invalid syntax (375068056.py, line 1)

: 

In [6]:
pip install insightface


  Using cached insightface-0.7.3-cp311-cp311-win_amd64.whl
  Using cached onnx-1.19.0-cp311-cp311-win_amd64.whl.metadata (7.2 kB)
  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached onnx-1.19.0-cp311-cp311-win_amd64.whl (16.5 MB)

   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]
   ---------------------------------------- 0/4 [onnx]



ERROR: Could not install packages due to an OSError: [WinError 206] Nom de fichier ou extension trop long: 'C:\\Users\\kokou\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\onnx\\backend\\test\\data\\node\\test_attention_3d_with_past_and_present_qk_matmul_softcap_expanded\\test_data_set_0'



In [5]:
from insightface.app import FaceAnalysis

app = FaceAnalysis(name=MODEL_NAME)
app.prepare(ctx_id=CTX_ID, det_size=DET_SIZE)

def embed_one_image(path: str):
    """
    Retourne l'embedding L2-normalisé (np.ndarray (512,)) ou None si aucun visage valide.
    """
    img = cv2.imread(path)
    if img is None:
        return None
    img_rgb = img[:, :, ::-1]  # BGR -> RGB
    faces = app.get(img_rgb)
    if not faces:
        return None
    f = max(faces, key=lambda x: x.det_score)
    return f.normed_embedding  # float32, déjà L2-normalisé


ModuleNotFoundError: No module named 'insightface'

## 4) Utilitaires : lecture train, split dev, extraction embeddings
- `read_train_pairs` lit les paires `XXXX_0`, `XXXX_1`.
- `split_train_dev` fait un split par identités.
- `embed_paths` calcule les embeddings d’une liste de chemins.


In [ ]:
def read_train_pairs(train_dir: Path):
    """
    Retourne une liste [(id_str, path0, path1), ...] pour les identités ayant exactement 2 images.
    """
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".webp")
    files = sorted([str(p) for p in train_dir.iterdir() if p.suffix.lower() in exts])
    by_id = {}
    for p in files:
        name = Path(p).stem  # e.g., 0123_0
        if "_" not in name:
            continue
        pid = name.rsplit("_", 1)[0]
        by_id.setdefault(pid, []).append(p)
    pairs = []
    for pid, paths in by_id.items():
        if len(paths) == 2:
            p0, p1 = sorted(paths)
            pairs.append((pid, p0, p1))
    return pairs

def split_train_dev(pairs, dev_ratio=0.2, seed=42):
    n = len(pairs)
    idx = np.arange(n)
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)
    cut = int(n * (1 - dev_ratio))
    return [pairs[i] for i in idx[:cut]], [pairs[i] for i in idx[cut:]]

def embed_paths(paths):
    """
    Calcule les embeddings pour une liste de chemins d'images.
    Retourne (embeddings_list, same_length_paths_list).
    """
    E, P = [], []
    for p in tqdm(paths, desc="Embeddings"):
        e = embed_one_image(p)
        E.append(None if e is None else e.astype(np.float32))
        P.append(p)
    return E, P


## 5) Évaluation sur split dev : ROC/EER + matching simulé
Permet de **valider** que la baseline se comporte bien avant de lancer le test.


In [ ]:
def pair_scores_dev(dev_images, dev_labels, dev_embeddings):
    """
    Construit des paires positives (même ID) et négatives (décalage simple),
    renvoie y_true (1/0) et y_score (similarités).
    """
    pos_scores, neg_scores = [], []
    # positives: (2i, 2i+1)
    for i in range(0, len(dev_images), 2):
        e1, e2 = dev_embeddings[i], dev_embeddings[i+1]
        if e1 is None or e2 is None:
            continue
        pos_scores.append(float(np.dot(e1, e2)))
    # négatives: croiser avec l'identité suivante
    for i in range(0, len(dev_images), 2):
        j = (i + 2) % len(dev_images)
        e1, e2 = dev_embeddings[i], dev_embeddings[j]
        if e1 is None or e2 is None:
            continue
        neg_scores.append(float(np.dot(e1, e2)))
    y_true = np.array([1]*len(pos_scores) + [0]*len(neg_scores))
    y_score = np.array(pos_scores + neg_scores)
    return y_true, y_score

def compute_eer(fpr, tpr, thresholds):
    fnr = 1 - tpr
    idx = np.nanargmin(np.abs(fnr - fpr))
    eer = (fnr[idx] + fpr[idx]) / 2
    return eer, thresholds[idx]

def simulate_matching(dev_embeddings, dev_labels):
    """
    Matching parfait simulé sur le split dev, renvoie (accuracy, nb_paires, liste_indices_paires).
    """
    valid_idx = [i for i, e in enumerate(dev_embeddings) if e is not None]
    if len(valid_idx) < 2:
        return 0.0, 0, []

    E = np.stack([dev_embeddings[i] for i in valid_idx], axis=0)
    S = E @ E.T
    np.fill_diagonal(S, -np.inf)
    cost = -S

    row, col = linear_sum_assignment(cost)
    used = set()
    pairs = []
    for r, c in zip(row, col):
        if r != c and r not in used and c not in used:
            pairs.append((r, c, float(S[r, c])))
            used.add(r); used.add(c)

    # accuracy: même ID ?
    idmap = {k: dev_labels[valid_idx[k]] for k in range(len(valid_idx))}
    correct = sum(1 for r, c, _ in pairs if idmap[r] == idmap[c])
    acc = correct / len(pairs) if pairs else 0.0
    return acc, len(pairs), [(valid_idx[r], valid_idx[c], s) for r, c, s in pairs]

# --- pipeline dev ---
pairs = read_train_pairs(TRAIN_DIR)
print(f"#IDs dans train : {len(pairs)}")
train_pairs, dev_pairs = split_train_dev(pairs, dev_ratio=DEV_RATIO, seed=SEED)
print(f"Split => train: {len(train_pairs)} | dev: {len(dev_pairs)}")

# construire le lot dev (images/labels)
dev_images, dev_labels = [], []
for pid, p0, p1 in dev_pairs:
    dev_images.extend([p0, p1])
    dev_labels.extend([pid, pid])

# embeddings dev
dev_embeddings, _ = embed_paths(dev_images)

# ROC/EER
if len(dev_images) >= 2:
    y_true, y_score = pair_scores_dev(dev_images, dev_labels, dev_embeddings)
    if len(np.unique(y_true)) == 2 and len(y_true) > 0:
        auc = roc_auc_score(y_true, y_score)
        fpr, tpr, thr = roc_curve(y_true, y_score)
        eer, thr_eer = compute_eer(fpr, tpr, thr)
        print(f"AUC = {auc:.4f} | EER ≈ {eer:.4f} @ thr = {thr_eer:.4f}")
        plt.figure()
        plt.plot(fpr, tpr, label=f"ROC (AUC={auc:.3f})")
        plt.plot([0, 1], [0, 1], "--")
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(); plt.title("ROC (dev)")
        if SAVE_FIG:
            Path(FIG_DIR).mkdir(parents=True, exist_ok=True)
            plt.savefig(Path(FIG_DIR) / "roc_dev.png", bbox_inches="tight", dpi=120)
        plt.show()
    else:
        print("Je ne peux pas tracer la ROC (pas assez de paires valides).")

# Matching simulé sur dev
acc, n_pairs, _pairs_idx = simulate_matching(dev_embeddings, dev_labels)
print(f"Matching simulé (dev) : {n_pairs} paires, accuracy = {acc:.4f}")


## 6) Inférence sur **test** + matching 1-à-1 + export CSV
Produit `outputs/tache1_submission.csv` au format :

image_A,image_B,similarity

imgA.jpg,imgB.jpg,0.812534

In [ ]:
# Lister les images test
exts = (".jpg", ".jpeg", ".png", ".bmp", ".webp")
test_paths = sorted([str(p) for p in Path(TEST_DIR).iterdir() if p.suffix.lower() in exts])
print("#images test :", len(test_paths))
assert len(test_paths) > 0, "Aucune image dans data/test."

# Embeddings test
test_embeddings, kept_paths = embed_paths(test_paths)

# Indices valides (visage détecté)
valid_idx = [i for i, e in enumerate(test_embeddings) if e is not None]
print("#images test valides (visage détecté) :", len(valid_idx))
if len(valid_idx) < 2:
    raise RuntimeError("Trop peu d'images test détectées. Augmente DET_SIZE ou mets ctx_id=-1 pour CPU.")

# Doit être pair pour appariement parfait
if len(valid_idx) % 2 == 1:
    print("⚠️ Nombre impair d'images valides : une image sera ignorée (score le plus faible au voisinage).")
    # on supprime l'indice avec la plus faible somme de similarité brute (approx)
    # calcule E provisoire
    E_tmp = np.stack([test_embeddings[i] for i in valid_idx], axis=0)
    S_tmp = E_tmp @ E_tmp.T
    np.fill_diagonal(S_tmp, 0.0)
    sums = S_tmp.sum(axis=1)
    drop_local = int(np.argmin(sums))
    drop_global = valid_idx[drop_local]
    valid_idx = [i for i in valid_idx if i != drop_global]

# Matrice de similarité
E = np.stack([test_embeddings[i] for i in valid_idx], axis=0)
S = E @ E.T
np.fill_diagonal(S, -np.inf)
cost = -S

# Hungarian
row, col = linear_sum_assignment(cost)
used = set()
pairs = []
for r, c in zip(row, col):
    if r != c and r not in used and c not in used:
        pairs.append((r, c, float(S[r, c])))
        used.add(r); used.add(c)

# Export CSV
export_path = Path(OUT_DIR) / EXPORT_NAME
export_path.parent.mkdir(parents=True, exist_ok=True)
with open(export_path, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["image_A", "image_B", "similarity"])
    for r, c, s in pairs:
        a = Path(kept_paths[valid_idx[r]]).name
        b = Path(kept_paths[valid_idx[c]]).name
        w.writerow([a, b, f"{s:.6f}"])

print("✅ CSV écrit :", export_path)
